In [ ]:
# Работа с сервером
import requests
from pprint import pprint
import json

# Распаковка и парсинг
import zipfile
import pandas as pd
import io
from bs4 import BeautifulSoup
from datetime import datetime


from urllib.parse import urljoin, urlparse
import os
import shutil
import base64
import importlib
import re

from pathlib import Path

In [10]:
# Загружаем словари для отправки Мain info
from functions import get_dict

ranks = get_dict('ranks')
gender = get_dict('gender')
marital_statuses = get_dict('marital_statuses')
resident_statuses = get_dict('resident_statuses')
languages = get_dict('languages')


In [ ]:
PATH = 'out/out_min'

In [ ]:
# Все HTML файлы из папки
html_files = [str(p) for p in Path(PATH).rglob('*.html')]


In [64]:
import functions
importlib.reload(functions)

for file in html_files:
    # получаем объект soup
    soup = functions.get_html_content(file)
    # парсим всю инфу
    main_info = functions.main_parser(soup)
    # и заметки
    notes = functions.parse_notes(soup)
    # собираем краткую инфу
    short_info = {k: main_info['Main info'][k] for k in ['Name / Surname:', 'Position applied for:', 'Desired Vessel Type:', 'Minimum salary $:']}
    # смотрим последний рейс
    last_voyage = main_info['Sea service (last 5 years)'][0] if main_info['Sea service (last 5 years)'] else None
    is_photo = f'Фото {"Есть" if functions.get_photo_simple(soup) else "Нет"}'
    # вывод резюме
    pprint(file)
    print(is_photo)
    pprint(short_info, sort_dicts=False)
    pprint(last_voyage)
    pprint(notes)
    print()
    

'out\\out_min\\seafarers_102.html'
Фото Нет
{'Name / Surname:': 'Sergiy Semenenko',
 'Position applied for:': ['2nd Officer'],
 'Desired Vessel Type:': ['Cargo vessels', 'Tankers', 'Offshore fleet'],
 'Minimum salary $:': '3000'}
None
('Голландские доки до 19.05.2015, 2-м 7 раз на карго, танкерах, АХТС, балк. '
 'танкерные ознаком.')

'out\\out_min\\seafarers_104588.html'
Фото Есть
{'Name / Surname:': 'omar shaban',
 'Position applied for:': ['3rd Officer', '2nd Officer'],
 'Desired Vessel Type:': ['Container carriers', 'Cargo vessels'],
 'Minimum salary $:': 'Negotiable'}
{'From - Till': '14.09.2017 - 10.02.2018',
 'ME Type / kW': 'MAK',
 'Position': '2nd Officer',
 'Shipowner / Country': 'Bassma shipping / Limassol',
 'Vessel Name / Flag': 'Mv/prince bassel / Panama',
 'Vessel type / DWT': 'General Cargo Vessel / 5000'}
("I'm hard worker having leadership ship and can work with any nationality and "
 'mixed crew on any type of vessel')

'out\\out_min\\seafarers_238919.html'
Фото Нет


In [ ]:
# soup = functions.get_html_content(file)

# seafarer_dict = {
#     # "photo": FILE, 
#     "name": functions.soup_extractor(soup,'Name / Surname:')[0].split(' ')[0],
#     # "middle_name": STRING,
#     "surname": functions.soup_extractor(soup,'Name / Surname:')[0].split(' ')[1],
#     'rank':functions.soup_extractor(soup,'Position applied for:')[0],
#     'aditional_ranks': functions.soup_extractor(soup,'Position applied for:')[1:],
    
#     # "rank_id": INTEGER,
#     # "additional_ranks_id": [int, int, int, ..],
#     # "calling_name": STRING,
#     # "available_from": DATE,
#     # "available_to": DATE,
#     # "date_of_birth": DATE,
#     # "place_of_birth": STRING,
#     'gender': functions.from_biometrics(soup,'Sex'),
#     # "gender_id": INTEGER,
#     'next_of_kin': functions.soup_extractor(soup,'Next of kin:')[0],
#     # "marital_status_id": INTEGER,
#     # "nationality_country_id": INTEGER,
#     # emails = functions.soup_extractor(soup,'E-mail:')
#     "emails": functions.soup_extractor(soup,'E-mail:')
#     #     {"email":"email1@mail.com", "comment":"comment", "uuid":null}
#     # ],
#     # "resident_status_id": INTEGER,
#     # "fast_note": STRING,
#     # "phone_numbers": [
#     #     {"uuid":null, "country_id":3, "number":"11223344", "type_id":1, "comment":"comment"}
#     # ],
#     # "personal_id": INTEGER,
#     # "language_id": INTEGER,
# }

# seafarer_dict

In [ ]:

# # ячейка для присвоения значений переменным
# photo = 
# name =
# middle_name =
# surname =

# rank_id = get_id(ranks,extract_anything('out\\out_min\\seafarers_104588.html', key))

# additional_ranks_id =
# calling_name =
# available_from =
# available_to =
# # date_of_birth =
# place_of_birth =
# gender_id =
# marital_status_id =
# nationality_country_id =
# emails =
# resident_status_id =
# fast_note =
# phone_numbers =
# personal_id =
# language_id =

# # Словарь формат json с основной информацией о моряке,
# # который будет отправлен для получения uuid моряка
# # - по нему в дальнейшем будет добавляться информация в БД
# main = {
#     "photo": photo #FILE, ?? 
    
#     "name": name #STRING,
#     "middle_name": middle_name #STRING,
#     "surname": surname #STRING,
#     "rank_id": rank_id #INTEGER,
#     "additional_ranks_id": additional_ranks_id #[int, int, int, ..],
#     "calling_name": calling_name #STRING,
#     "available_from": available_from #DATE,
#     "available_to": available_to #DATE,
#     "date_of_birth": date_of_birth #DATE,
#     "place_of_birth": place_of_birth #STRING,
#     "gender_id":gender_id # INTEGER,
#     "marital_status_id":marital_status_id # INTEGER,
#     "nationality_country_id":nationality_country_id # INTEGER,
#     "emails":emails # [
#         # {"email":"email1@mail.com", "comment":"comment", "uuid":null}
#     # ],
#     "resident_status_id": resident_status_id #INTEGER,
#     "fast_note": fast_note #STRING,  
#     "phone_numbers": phone_numbers #[
#         # {"uuid":null, "country_id":3, "number":"11223344", "type_id":1, "comment":"comment"}
#     # ],
#     "personal_id": personal_id #INTEGER,
#     "language_id": language_id #INTEGER,
# }

In [ ]:
# # определение url ссылок для использования

# # основной домен
# domain = 'https://staffdev.crew-man.com/api/v1/'

# # Добавление основной инфы моряка:POST
# main_info = 'seafarers/main'

# # seafarer_uuid = response_main.text
# # Добавление адресов моряка:POST
# # adress = f'seafarers/{seafarer_uuid}/addresses'

# # Добавление сертификатов (документов) моряка:POST
# # certificates = f'seafarers/{seafarer_uuid}/certificates'

# # Добавление родственников (kins) моряка:POST
# # relatives = f'seafarers/{seafarer_uuid}/relatives'

# # Добавление sea service моряка:POST
# contracts =  'contracts'

# route = main_info
# url = domain + route


In [ ]:
# from bs4 import MarkupResemblesLocatorWarning
# import warnings
# warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)

In [ ]:
# post ={
#     "photo": FILE, ??
#     "name": NAME,
#     "middle_name": STRING,
#     "surname": SURNAME,
#     "rank_id": INTEGER,
#     # "additional_ranks_id": [int, int, int, ..],
#     "calling_name": STRING,
#     "available_from": DATE,
#     "available_to": DATE,
#     "date_of_birth": DATE_OF_BIRTH,
#     "place_of_birth": STRING,
#     "gender_id": INTEGER,
#     "marital_status_id": INTEGER,
#     "nationality_country_id": INTEGER,
#     "emails": [
#         {"email":"email1@mail.com", "comment":"comment", "uuid":null}
#     ],
#     "resident_status_id": INTEGER,
#     "fast_note": STRING,
#     "phone_numbers": [
#         {"uuid":null, "country_id":3, "number":"11223344", "type_id":1, "comment":"comment"}
#     ],
#     "personal_id": INTEGER,
#     "language_id": INTEGER,
# }

In [ ]:
# main = {
    
#     "name": [name.strip() for name in 'Name / Surname:'.split('/')][0],
    
#     "surname": [name.strip() for name in 'Name / Surname:'.split('/')][1][:-1],
    
#     "rank_id": INTEGER, я так понимаю это ид должностей где они беруться?
#     "additional_ranks_id": [int, int, int, ..],
    
    
#     "date_of_birth": DATE,
#     # "place_of_birth": STRING,
    
#     "emails": [
#         {"email":"email1@mail.com", "comment":"comment", "uuid":null}
#     ],
#     "resident_status_id": INTEGER, резидент РФ?
#     "fast_note": STRING,
#     "phone_numbers": [
#         {"uuid":null, "country_id":3, "number":"11223344", "type_id":1, "comment":"comment"}
#     ],
#     "personal_id": INTEGER, где берется?
#     "language_id": INTEGER, где берется?
# }